# Parse DrugBank XML to extract drug product data

In [1]:
import collections
import gzip
import os.path
import pandas as pd
import xml.etree.ElementTree as ET

In [2]:
'''
Helper Functions
'''

def collapse_list_values(row):
    for key, value in row.items():
        if isinstance(value, list):
            row[key] = '|'.join(value)
    return row

def parse_xml(path):
    ext = os.path.splitext(path)[1]
    if ext == '.xml':
        print('Input type: xml')
        tree = ET.parse(db_file)
        return tree.getroot()
    elif ext == '.gzip':
        print('Input type: xml.gzip')
        with gzip.open(path) as xml_file:
            tree = ET.parse(xml_file)
        return tree.getroot()

In [3]:
# parse drugbank XML file (www.drugbank.ca/releases/5-1-4/downloads/all-full-database)
db_file = 'full_database.xml'
root = parse_xml(db_file)

Input type: xml


In [5]:
#namespace
ns = '{http://www.drugbank.ca}'

# product list to be populated
products = list()

# parse xml tree
drugs = root.getchildren()
for drug in drugs:
    dname = drug.findtext(ns + "name") # drug name
    dtype = drug.get('type') # drug type (e.g. small molecule, biotec etc.)
    db_id = drug.findtext(ns + "drugbank-id[@primary='true']")
    prod_list = drug.findall('{ns}products'.format(ns = ns))
    for prod in prod_list:
        prods = prod.getchildren()
        for p in prods:
            # dictionary to populate product info (each row is a product)
            row = collections.OrderedDict()
            row['drug_name'] = dname
            row['drug_type'] = dtype
            row['drugbank_id'] = db_id
            row['product_name'] = p.find("{ns}name".format(ns = ns)).text
            row['labeller'] = p.find("{ns}labeller".format(ns = ns)).text
            row['ndc_id'] = p.find("{ns}ndc-id".format(ns = ns)).text
            row['ndc_product_code'] = p.find("{ns}ndc-product-code".format(ns = ns)).text
            row['dpd_id'] = p.find("{ns}dpd-id".format(ns = ns)).text
            row['marketing_start'] = p.find("{ns}started-marketing-on".format(ns = ns)).text
            row['marketing_end'] = p.find("{ns}ended-marketing-on".format(ns = ns)).text
            row['dosage_form'] = p.find("{ns}dosage-form".format(ns = ns)).text
            row['strength'] = p.find("{ns}strength".format(ns = ns)).text
            row['route'] = p.find("{ns}route".format(ns = ns)).text
            row['fda_appl_no'] = p.find("{ns}fda-application-number".format(ns = ns)).text
            row['generic'] = p.find("{ns}generic".format(ns = ns)).text
            row['otc'] = p.find("{ns}over-the-counter".format(ns = ns)).text
            row['approved'] = p.find("{ns}approved".format(ns = ns)).text
            row['country'] = p.find("{ns}country".format(ns = ns)).text
            row['source'] = p.find("{ns}source".format(ns = ns)).text
            products.append(row)
print('finished parsing xml tree')

finished parsing xml tree


In [8]:
db_products = list(map(collapse_list_values, products))
cols = ['drug_name', 'drug_type', 'drugbank_id', 'product_name', 'labeller', 'ndc_id', 'ndc_product_code', 'dpd_id', 'marketing_start', 'marketing_end', 'dosage_form', 'strength', 'route', 'fda_appl_no', 'generic', 'otc', 'approved', 'country', 'source']
df = pd.DataFrame.from_dict(db_products)[cols]

In [10]:
df.to_csv('drugbank_products.csv', index=None, header=True)
print('total products: %s' % df.shape[0])

total products: 298383
